In [1]:
import pandas as pd
import sqlite3
import plotly.graph_objects as go
import numpy as np

In [34]:
connection = sqlite3.connect("../../data/checking-logs.sqlite")
timestamp = pd.read_sql_query(
  '''
  SELECT uid, date(timestamp) as date, COUNT(*) as commits
  FROM checker 
  WHERE uid LIKE "user_%" AND labname == "project1" AND status = "ready"
  group by uid, date
  order by date
  ''', connection, parse_dates=['date']
)

connection.close()

timestamp

,uid,date,commits
0,user_4,2020-04-17,7
1,user_17,2020-04-18,1
2,user_4,2020-04-19,4
3,user_30,2020-04-22,1
4,user_30,2020-04-23,1
...,...,...,...
89,user_4,2020-05-14,27
90,user_6,2020-05-14,1
91,user_8,2020-05-14,8
92,user_19,2020-05-15,4


In [40]:
data = timestamp.pivot_table(
    index='uid',
    columns='date',
    values='commits',
    aggfunc='sum',
    fill_value=0
)

data = data.cumsum(axis=1)

data

date,2020-04-17,2020-04-18,2020-04-19,2020-04-22,2020-04-23,2020-04-24,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15
uid,,,,,,,,,,,,,,,,,,,
user_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,11
user_10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,21,58,58
user_11,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1
user_12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4
user_13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,30,30,32,32
user_14,0,0,0,0,0,0,0,0,1,1,1,1,18,25,49,92,92,99,99
user_15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3
user_16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,9,9
user_17,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,3,5,5


In [50]:
days = data.columns.tolist()  
users = data.index.tolist()   


x_init = np.array([0])
initial_data = []
for user in users:
    y_axis = np.array([data.loc[user, days[0]]])  
    initial_data.append(go.Scatter(x=x_init,y=y_axis,mode="lines+markers",name=user))

frames = []
for f in range(0, len(days)):
    x_axis = np.arange(1, f+2)
    curr_data = []
    
    for user in users:
        y_axis = data.loc[user, days[:f+2]].values.astype(int)
        curr_data.append(go.Scatter(x=x_axis,y=y_axis,mode="lines+markers",name=user))
    frames.append(go.Frame(data=curr_data))

xaxis_range = [1, len(days) + 1]
y_max = data.values.max() + 10

figure = go.Figure(
    data=initial_data,
    layout={
        "title": "Dynamics of commits per user in project1",
        "xaxis": {"range": xaxis_range,"tickvals": np.arange(0, len(days)+1, 2),"ticktext": [d for d in range(len(days), 1)],'showgrid': True, "gridcolor": "white"},
        "yaxis": {"range": [0, y_max],'nticks': 10, 'showgrid': True, "gridcolor": "white",},
        "plot_bgcolor": "lightgrey",
        "updatemenus": [{"type": "buttons","buttons": [{"method": "animate","label": "Play","args": [None, {"frame": {"duration": 500, "redraw": True},"fromcurrent": True}]}]}]},
    frames=frames
)

figure.show()